<a href="https://colab.research.google.com/github/SaraBCoutinho/Pesquisa_Mestrado/blob/main/monol%C3%ADticos_predi%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
%%capture
!pip install datasets
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy import load
from wordcloud import STOPWORDS, WordCloud
!pip install ftfy
import ftfy
import nltk
import json
import re
%matplotlib inline
import csv
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
np.random.seed(0)

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV,cross_validate, StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import Binarizer, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import tree
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from collections import Counter, defaultdict
from os import listdir, makedirs
from os.path import isfile, join, splitext, split
from unicodedata import normalize
import nltk
nltk.download('stopwords')
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from gensim.models import Word2Vec

stopwords_list = stopwords.words('english')
import re
import string
!pip install zeugma
!pip install preprocessing
!pip install function
from zeugma.embeddings import EmbeddingTransformer
from preprocessing import *
from function import *
import itertools

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import time

### Data

In [ ]:
%%capture
!mkdir Data
!mkdir Data/liar
dataset = load_dataset('liar')
train, val, test = pd.DataFrame(dataset['train']), pd.DataFrame(dataset['validation']), pd.DataFrame(dataset['test'])

### Pré-processamento

In [ ]:
%%time
stopwords_list = stopwords.words('english')
def clean_text(text):
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",re.MULTILINE | re.UNICODE)
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
    text = re_url.sub("URL", text)
    text = re_ip.sub("IPADDRESS", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
#    text = " ".join(stemmed_words)
#    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in stemmed_words if not w in stops and len(w) >= 3]
    text = " ".join(text)
    return text

def tokenizer(text, flg_stemm=True, flg_lemm=False, lst_stopwords=stopwords_list):
    lst_text = text.lower().split()

    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    text = " ".join(lst_text)
    return text

CPU times: user 1.29 ms, sys: 0 ns, total: 1.29 ms
Wall time: 1.27 ms


In [ ]:
def preprocess_applying(train,val,test):

  train, val, test = train, val, test

  train['statement'], val['statement'], test['statement'] = train['statement'].map(clean_text), val['statement'].map(clean_text), test['statement'].map(clean_text)
  c_train, c_val, c_test = Counter(), Counter(), Counter()
  train_to_count, test_to_count, val_to_count = train.copy(), test.copy(), val.copy()

  def CountOnly(sentence): #funçao para contar os valores unicos
    sentence=sentence.split()
    result = Counter(sentence)
    return result

  train_to_count['statement'], val_to_count['statement'], test_to_count['statement'] = train_to_count['statement'].apply(CountOnly), val_to_count['statement'].apply(CountOnly), test_to_count['statement'].apply(CountOnly)

  for sentence in train_to_count['statement']:
    c_train = c_train + sentence
  for sentence in val_to_count['statement']:
    c_val = c_val + sentence
  for sentence in test_to_count['statement']:
    c_test = c_test + sentence

  list_unique_train, list_unique_val, list_unique_test, total_train, total_val, total_test = [], [], [], 0, 0, 0

  for k,v in c_train.items():
    total_train = total_train + v
    if v==1:
      list_unique_train.append(k)
  for k,v in c_val.items():
    total_val = total_val + v
    if v==1:
      list_unique_val.append(k)
  for k,v in c_test.items():
    total_test = total_test + v
    if v==1:
      list_unique_test.append(k)

  def cleanningOnceTrain(statement):
    statement=statement.split()
    result = [word for word in statement if word not in list_unique_train]
    result = " ".join(result)
    return result
  def cleanningOnceVal(statement):
    statement=statement.split()
    result = [word for word in statement if word not in list_unique_val]
    result = " ".join(result)
    return result
  def cleanningOnceTest(statement):
    statement=statement.split()
    result = [word for word in statement if word not in list_unique_test]
    result = " ".join(result)
    return result

  train['statement'] = train['statement'].apply(cleanningOnceTrain) #aplicando ao conjunto de dados
  val['statement'] = val['statement'].apply(cleanningOnceVal)
  test['statement'] = test['statement'].apply(cleanningOnceTest)

  return train, val, test

In [ ]:
train, val, test = preprocess_applying(train,val,test)
train, val, test = train.copy(), val.copy(), test.copy()
X_train, X_val, X_test = train['statement'], val['statement'], test['statement']
y_train, y_val, y_test = train['label'], val['label'], test['label']

train.to_csv('/content/Data/liar/train.csv')
val.to_csv('/content/Data/liar/val.csv')
test.to_csv('/content/Data/liar/test.csv')
y_train.to_csv('/content/Data/liar/train_labels.csv')
y_val.to_csv('/content/Data/liar/val_labels.csv')
y_test.to_csv('/content/Data/liar/test_labels.csv')

### Embeddings

In [ ]:
w2v = EmbeddingTransformer('word2vec')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
glove = EmbeddingTransformer('glove')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
fasttext = EmbeddingTransformer('fasttext')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')

## PARTE II

### Classificadores

#### Funções

In [ ]:
#--Classifiers
#--SVM
def get_svm(cv, tfidf, w2v, glove, fasttext):
    return{
        'CV': {'CLF': SVC(random_state=42, kernel='linear', gamma=0.1, probability=True),'EXT': cv,},
        'TFIDF': {'CLF': SVC(random_state=42, kernel='linear',
                             gamma=0.1, probability=True),'EXT': tfidf,},
        'W2V': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': w2v,},
        'GLOVE': {'CLF': SVC(random_state=42, kernel='rbf', gamma=0.5, probability=True),'EXT':  glove,},
        'FAST': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': fasttext,}
    }
#--LR
def get_lr(cv,tfidf,glove,w2v,fasttext):
    return {
        'CV': {'CLF': LogisticRegression(random_state=42,multi_class='auto', solver='liblinear', penalty='l1'),'EXT': cv,},
        'TFIDF': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': tfidf,},
        'GLOVE': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT':  glove,},
        'W2V': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': w2v,},
        'FAST': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': fasttext,}
        }
#--RF
def get_rf(cv, tfidf, w2v, glove, fasttext):
    return {'CV': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=20, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
            }
#--NB
def get_nb(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': MultinomialNB(alpha=1, fit_prior=False),'EXT': cv,},
        'TFIDF': {'CLF': MultinomialNB(alpha=0.5, fit_prior=False),'EXT': tfidf,},
        'W2V': {'CLF': BernoulliNB(alpha=0.5, fit_prior=True),'EXT': w2v,},
        'GLOVE': {'CLF': BernoulliNB(alpha=0.1, fit_prior=True),'EXT':  glove,},
        'FAST': {'CLF': BernoulliNB(alpha=1, fit_prior=True),'EXT': fasttext,}
    }
#--MLP
def get_mlp(cv, tfidf, w2v, glove, fasttext):
    return {'CV': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40,activation='relu', solver='lbfgs'),'EXT': cv,},
        'TFIDF': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40,activation='logistic', solver='adam'),'EXT': tfidf,},
        'W2V': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': w2v,},
        'GLOVE': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT':  glove,},
        'FAST': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': fasttext,}
    }
#--EXTRA
def get_extra(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
    }
#--KNN
def get_knn(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': KNeighborsClassifier(n_neighbors=3, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': fasttext,}
    }

def get_classifier(clf, X, y, ext):
    pipe_clf = Pipeline([('extractor', ext),('clf', clf)])
    pipe_clf.fit(X, y)
    return pipe_clf

def get_CNN_6(ext, tokenizer, embedding_matrix, MAX_NB_WORDS, EMBEDDING_DIM=300, MAX_SEQUENCE_LENGTH=300, activation='sigmoid', word_embedding=False, dense=6):
    word_index = tokenizer.word_index
    nb_words = min(MAX_NB_WORDS, len(word_index)) + 1

    if ext == cv:
      embedding_matrix = embedding_matrix_cv
    elif ext == tfidf:
      embedding_matrix = embedding_matrix_tfidf
    if word_embedding == True:
      model = ext.model
      embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
      for word, i in word_index.items():
        if word in model.key_to_index:
          embedding_matrix[i] = model.get_vector(word, norm=True)

    embedding_layer = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False
                                )
    cnn = Sequential()
    cnn.add(embedding_layer)
    cnn.add(Dropout(0.2))
    cnn.add(Conv1D(64, 6, padding='valid', activation='relu', strides=1))
    cnn.add(GlobalMaxPooling1D())
    cnn.add(Dense(256))
    cnn.add(Dropout(0.2))
    cnn.add(Activation('relu'))
    cnn.add(Dense(6))
    cnn.add(Activation(activation))
    cnn.summary()
    cnn.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return cnn

def update_pred_proba_6(clfs, label, train, val, test):
    pred_train = pd.DataFrame()
    pred_val = pd.DataFrame()
    pred_test = pd.DataFrame()

    prob_train = pd.DataFrame()
    prob_val = pd.DataFrame()
    prob_test = pd.DataFrame()

    for ext, clf in clfs.items():
        # Predict
        df_pred_train_ = pd.DataFrame(clf.predict(train), columns=["{}-{}".format(ext,label)])
        df_pred_val_ = pd.DataFrame(clf.predict(val), columns=["{}-{}".format(ext,label)])
        df_pred_test_ = pd.DataFrame(clf.predict(test), columns=["{}-{}".format(ext,label)])

        # # Probabilidades
        cols = [
            "{}-{}-{}".format(ext,label, clf.classes_[0]),
            "{}-{}-{}".format(ext,label, clf.classes_[1]),
            "{}-{}-{}".format(ext,label, clf.classes_[2]),
            "{}-{}-{}".format(ext,label, clf.classes_[3]),
            "{}-{}-{}".format(ext,label, clf.classes_[4]),
            "{}-{}-{}".format(ext,label, clf.classes_[5])
        ]
        df_prob_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
        df_prob_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
        df_prob_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)

        pred_train = pd.concat([pred_train, df_pred_train_], axis=1, sort=False)
        pred_val = pd.concat([pred_val, df_pred_val_], axis=1, sort=False)
        pred_test = pd.concat([pred_test, df_pred_test_], axis=1, sort=False)

        prob_train = pd.concat([prob_train, df_prob_train_], axis=1, sort=False)
        prob_val = pd.concat([prob_val, df_prob_val_], axis=1, sort=False)
        prob_test = pd.concat([prob_test, df_prob_test_], axis=1, sort=False)

    return  pred_train, pred_val, pred_test, prob_train, prob_val, prob_test

#### Classificação  

In [ ]:
%%time
w2v.fit_transform(X_train)
glove.fit_transform(X_train)
fasttext.fit_transform(X_train)
cv.fit_transform(X_train)
tfidf.fit_transform(X_train)
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(train['statement'])
word_index = tokenizer.word_index
print(len(tokenizer.word_index))

5004
CPU times: user 2.51 s, sys: 22.4 ms, total: 2.53 s
Wall time: 2.6 s


In [ ]:
MAX_NB_WORDS = 20000
nb_words = min(MAX_NB_WORDS, len(word_index)) + 1
EMBEDDING_DIM=300
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

embedding_matrix_cv = embedding_matrix #cv
X_ext_cv = cv.get_feature_names_out()
model_cv = Word2Vec(sentences=X_ext_cv, min_count=1, workers=1, vector_size=300)
for word, i in word_index.items():
    if word in model_cv.wv.index_to_key:
        embedding_matrix_cv[i] = model_cv.wv.word_vec(word)

embedding_matrix_tfidf = embedding_matrix #tfidf
X_ext_tfidf = tfidf.get_feature_names_out()
model_tfidf = Word2Vec(sentences=X_ext_tfidf, min_count=1, workers=1, vector_size=300)
for word, i in word_index.items():
    if word in model_tfidf.wv.index_to_key:
        embedding_matrix_tfidf[i] = model_tfidf.wv.word_vec(word)


In [ ]:
%%time
def main():
  for db in ["liar"]:
    if db == 'liar':
      classe = 'label'
      df_pred_train = pd.DataFrame()
      df_pred_val = pd.DataFrame()
      df_pred_test = pd.DataFrame()
      df_prob_val = pd.DataFrame()
      df_prob_train = pd.DataFrame()
      df_prob_test = pd.DataFrame()

      df_pred_train = pd.concat([df_pred_train, train['label']], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, val['label']], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, test['label']], axis=1, sort=False)
      df_prob_train = pd.concat([df_prob_train, train['label']], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, val['label']], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, test['label']], axis=1, sort=False)
      print('dados ok')

      # SVM
      classifier = get_svm(cv, tfidf, w2v, glove, fasttext)
      clfs = { }
      for ext, clf in classifier.items():
        ini = time.time()
        clfs[ext] = get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])
        fim = time.time()
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_6(clfs, "SVM", X_train, X_val, X_test)
      print('svm parte 1 ok e tempo de treino: ', fim-ini)

      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)
      print('svm parte 2 ok')

      # LR
      classifier = get_lr(cv,tfidf,glove,w2v,fasttext)
      for ext, clf in classifier.items():
        ini = time.time()
        clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})
        fim = time.time()
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_6(clfs, "LR", X_train, X_val, X_test)
      print('lr ok e tempo de treino: ', fim-ini)

      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)
      print('lr parte 2 ok')

      # RF
      classifier = get_rf(cv, tfidf, w2v, glove, fasttext)
      for ext, clf in classifier.items():
        ini = time.time()
        clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})
        fim = time.time()
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_6(clfs, "RF", X_train, X_val, X_test)
      print('rf ok e tempo de treino: ', fim-ini)

      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)
      print('rf parte 2 ok')

      # NB
      classifier = get_nb(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
        ini = time.time()
        clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})
        fim = time.time()
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_6(clfs, "NB", X_train, X_val, X_test)
      print('nb ok e tempo de treino: ', fim-ini)

      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)
      print('nb parte 2 ok')

      # MLP
      classifier = get_mlp(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
        ini = time.time()
        clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})
        fim = time.time()
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_6(clfs, "MLP", X_train, X_val, X_test)
      print('mlp ok e tempo de treino: ', fim-ini)

      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)
      print('mlp parte 2 ok')

      # Extra
      classifier = get_extra(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
        ini = time.time()
        clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})
        fim = time.time()
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_6(clfs, "EXTRA", X_train, X_val, X_test)
      print('extra ok e tempo de treino: ', fim-ini)

      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)
      print('extra parte 2 ok')

      # KNN
      classifier = get_knn(cv, tfidf, w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
        ini = time.time()
        clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})
        fim = time.time()
      pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_6(clfs, "KNN", X_train, X_val, X_test)
      print('knn ok e tempo de treino: ', fim-ini)

      df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

      df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)
      print('knn parte 2 ok')

      #CNN
      MAX_NB_WORDS = 20000
      MAX_SEQUENCE_LENGTH=300
      tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
      tokenizer.fit_on_texts(X_train)
      seq_train = tokenizer.texts_to_sequences(X_train)
      seq_val = tokenizer.texts_to_sequences(X_val)
      seq_test = tokenizer.texts_to_sequences(X_test)
      data_train = pad_sequences(seq_train,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_val = pad_sequences(seq_val,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_test = pad_sequences(seq_test, padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)

      cnn_cv = get_CNN_6(cv, tokenizer, None, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', dense=6)
      cnn_tfidf = get_CNN_6(tfidf, tokenizer,  None, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', dense=6)
      cnn_w2v = get_CNN_6(w2v, tokenizer,  None, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=6)
      cnn_glove = get_CNN_6(glove, tokenizer,  None, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True, dense=6)
      cnn_fast = get_CNN_6(fasttext, tokenizer,  None, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=6)
      y_validation=to_categorical(y_val)
      y_training=to_categorical(y_train)
      y_testing=to_categorical(y_test)


      ini = time.time()
      cnn_cv.fit(data_train, y_training, validation_data=(data_val,y_validation), epochs=20, batch_size=20)
      cnn_tfidf.fit(data_train, y_training, validation_data=(data_val,y_validation), epochs=20, batch_size=20)
      cnn_w2v.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_glove.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_fast.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      fim = time.time()
      print('cnn ok e tempo de treino: ', fim-ini)

      cols = ["CNN-CV-0", "CNN-CV-1","CNN-CV-2", "CNN-CV-3","CNN-CV-4", "CNN-CV-5"]
      df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
      df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
      df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

      df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-TFIDF-0", "CNN-TFIDF-1","CNN-TFIDF-2", "CNN-TFIDF-3","CNN-TFIDF-4", "CNN-TFIDF-5"]
      df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
      df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
      df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

      df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-W2V-0", "CNN-W2V-1","CNN-W2V-2", "CNN-W2V-3","CNN-W2V-4", "CNN-W2V-5"]
      df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
      df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
      df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

      df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-GLOVE-0", "CNN-GLOVE-1","CNN-GLOVE-2", "CNN-GLOVE-3","CNN-GLOVE-4", "CNN-GLOVE-5"]
      df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
      df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
      df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

      df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-FAST-0", "CNN-FAST-1","CNN-FAST-2", "CNN-FAST-3","CNN-FAST-4", "CNN-FAST-5"]
      df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
      df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
      df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])
      df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

      df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
      df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)
      df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)
      print('cnn parte 2 ok')

      path = "./Data/{}".format(db)
      df_pred_train.to_csv("{}/pred_train.csv".format(path))
      df_pred_val.to_csv("{}/pred_val.csv".format(path))
      df_pred_test.to_csv("{}/pred_test.csv".format(path))

      df_prob_train.to_csv("{}/prob_train.csv".format(path))
      df_prob_val.to_csv("{}/prob_val.csv".format(path))
      df_prob_test.to_csv("{}/prob_test.csv".format(path))


CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 13.6 µs


In [ ]:
%%time
if __name__ == "__main__":
    main()

dados ok
svm parte 1 ok e tempo de treino:  263.0845401287079
svm parte 2 ok
lr ok e tempo de treino:  11.0009183883667
lr parte 2 ok
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 1 of 20
building tree 2 of 20
building tree 3 of 20[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.3s

building tree 4 of 20[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s

building tree 5 of 20[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.5s

building tree 6 of 20[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.5s

building tree 7 of 20[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.8s

building tree 8 of 20[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.8s

building tree 9 of 20[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    1.0s

building tree 10 of 20[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.0s

building tree 11 of 20[Parallel(n_jobs=-1)]: Done   9 tasks      | e